In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using QMDP
using JLD


(process:4496): Gtk-WARNING **: 21:59:19.210: Locale not supported by C library.
	Using the fallback 'C' locale.


In [2]:
p = JLD.load("QMDP_Bumper_policy.jld")
p = p["policy"]

┌ Warning: type BasicPOMCP.POMCPPlanner{AA228FinalProject.RoombaPOMDP{AA228FinalProject.Bumper,Core.Bool},BasicPOMCP.SolvedPORollout{POMDPPolicies.RandomPolicy{Random.MersenneTwister,AA228FinalProject.RoombaPOMDP{AA228FinalProject.Bumper,Core.Bool},BeliefUpdaters.NothingUpdater},BeliefUpdaters.NothingUpdater,Random.MersenneTwister},Random.MersenneTwister} not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type BasicPOMCP.POMCPSolver not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type MCTS.ExceptionRethrow not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type MCTS.RolloutEstimator not present in workspace; reconstructing
└ @ JLD /Users/shengji/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type BasicPOMCP.SolvedPORollout{POMDPPolicies.RandomPolicy{Random.Mersen

getfield(JLD, Symbol("##BasicPOMCP.POMCPPlanner{AA228FinalProject.RoombaPOMDP{AA228FinalProject.Bumper,Core.Bool},BasicPOMCP.SolvedPORollout{POMDPPolicies.RandomPolicy{Random.MersenneTwister,AA228FinalProject.RoombaPOMDP{AA228FinalProject.Bumper,Core.Bool},BeliefUpdaters.NothingUpdater},BeliefUpdaters.NothingUpdater,Random.MersenneTwister},Random.MersenneTwister}#360"))(getfield(JLD, Symbol("##BasicPOMCP.POMCPSolver#367"))(20, 1.0, 1000, Inf, false, getfield(JLD, Symbol("##MCTS.ExceptionRethrow#376"))(), MersenneTwister(UInt32[0x0634ab2c, 0xcccf14e6, 0x3d8fa6b0, 0xd3673877], Random.DSFMT.DSFMT_state(Int32[-1136388431, 1073288361, -2131327580, 1072748135, -1377061484, 1073012656, -1974709828, 1073212938, 391128126, 1072898135  …  646644001, 1073669527, 1688961526, 1073628475, 92118486, 2126273365, -1666819623, 1912510256, 382, 0]), [1.93443, 1.35969, 1.26592, 1.0276, 1.72788, 1.31693, 1.05031, 1.2343, 1.54865, 1.29179  …  1.24471, 1.53711, 1.96345, 1.51673, 1.07636, 1.83763, 1.83812, 1.

In [3]:
num_x_pts = 50 # e.g. 50
num_y_pts = 50 # e.g. 50
num_th_pts = 20 # e.g. 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);

In [4]:
vlist = [0, 1, 2, 3]
omlist = [-1, -0.5, 0, 0.5, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [5]:
sensor = Bumper();

In [6]:
config = 3 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));

In [6]:
num_particles = 2000
resampler = LidarResampler(num_particles, LowVarianceResampler(num_particles))
# for the bumper environment
# resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [7]:
using POMDPSolve

┌ Info: Recompiling stale cache file /Users/shengji/.julia/compiled/v1.0/POMDPSolve/gD8mK.ji for POMDPSolve [a575d7f4-a5df-53f5-8ba4-9d691c1ba8ff]
└ @ Base loading.jl:1190


In [8]:
solver = POMDPSolveSolver()

POMDPSolveSolver(Dict{AbstractString,Any}())

In [9]:
p = solve(solver, m)

MethodError: MethodError: no method matching solve(::POMDPSolveSolver, ::RoombaPOMDP{Bumper,Bool})
Closest candidates are:
  solve(::POMDPSolveSolver, !Matched::POMDPSolveFile) at /Users/shengji/.julia/packages/POMDPSolve/JTyCb/src/solver.jl:177
  solve(::POMDPSolveSolver, !Matched::POMDPSolveFile, !Matched::POMDPSolvePolicy) at /Users/shengji/.julia/packages/POMDPSolve/JTyCb/src/solver.jl:177
  solve(!Matched::FunctionSolver, ::Union{MDP, POMDP}) at /Users/shengji/.julia/packages/POMDPPolicies/nVH68/src/function.jl:23
  ...

In [8]:
solver = QMDPSolver(max_iterations=20, tolerance=1e-3, verbose=true)
p = solve(solver, m);

[Iteration 1   ] residual:        0.1 | iteration runtime:  34554.668 ms, (      34.6 s total)
[Iteration 2   ] residual:      0.095 | iteration runtime:  88547.387 ms, (       123 s total)


InterruptException: InterruptException:

In [9]:
# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

In [15]:
using JLD
JLD.save("QMDP_Bumper_policy.jld", "policy", p)

In [12]:
using POMDPFiles

In [42]:
fout = open("aaa.pomdp", "w")
write(fout, m)
close(fout)

MethodError: MethodError: no method matching reward(::RoombaPOMDP{Bumper,Bool}, ::RoombaState, ::RoombaAct)
Closest candidates are:
  reward(::Union{RoombaMDP, RoombaPOMDP}, ::AbstractArray{Float64,1}, ::AbstractArray{Float64,1}, !Matched::AbstractArray{Float64,1}) at /Users/shengji/Documents/GitHub/Room-Navigation/src/roomba_env.jl:347
  reward(::Union{MDP, POMDP}, ::Any, ::Any, !Matched::Any) at /Users/shengji/.julia/packages/POMDPs/JiYXY/src/pomdp.jl:107
  reward(!Matched::POMDPModelTools.FullyObservablePOMDP{S,A}, ::S, ::A) where {S, A} at /Users/shengji/.julia/packages/POMDPModelTools/eHEjm/src/fully_observable_pomdp.jl:41
  ...

In [19]:
using POMDPModels

┌ Info: Recompiling stale cache file /Users/shengji/.julia/compiled/v1.0/POMDPModels/GHWgR.ji for POMDPModels [355abbd5-f08e-5560-ac9e-8b5f2592a0ca]
└ @ Base loading.jl:1190
┌ Info: Loading Cairo backend into Compose.jl
└ @ Compose /Users/shengji/.julia/packages/Compose/BYWXX/src/Compose.jl:161
┌ Warning: Package Compose does not have Cairo in its dependencies:
│ - If you have Compose checked out for development and have
│   added Cairo as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Compose
│ Loading Cairo into Compose from project dependency, future warnings for Compose are suppressed.
└ @ nothing nothing:840


In [20]:
a = TigerPOMDP()

TigerPOMDP(-1.0, -100.0, 10.0, 0.85, 0.95)

In [21]:
typeof(a)

TigerPOMDP

In [22]:
m

RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 3
  sspace: DiscreteRoombaStateSpace
  room: AA228FinalProject.Room
  aspace: Array{RoombaAct}((20,))
  _amap: Dict{RoombaAct,Int64}
)